In [95]:
import src.classifiers.linear as linear
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
results = linear.main('./data/pair_attacks.npz')

Data loaded: 432
Hyperparam optimization...


  6%|████████▌                                                                                                                                                | 1/18 [00:26<07:23, 26.06s/it]

new best:  0.4772727272727273 l1 0.001 0


 11%|█████████████████                                                                                                                                        | 2/18 [00:27<03:02, 11.38s/it]

new best:  0.5227272727272727 l1 0.002 0


 22%|██████████████████████████████████                                                                                                                       | 4/18 [00:50<02:55, 12.57s/it]


KeyboardInterrupt: 

/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [96]:
X, y = linear.extract_data(linear.load_as_npz('./data/pair_attacks.npz'))

In [102]:
len(X.sum(axis=0).nonzero()[0])

11001

In [94]:
stronger_results = linear.main('./data/pair_attacks.npz')

Data loaded: 312
Hyperparam optimization...


  6%|████████▌                                                                                                                                                | 1/18 [00:00<00:03,  4.75it/s]

new best:  0.5 l1 0.001 0


 28%|██████████████████████████████████████████▌                                                                                                              | 5/18 [00:04<00:16,  1.30s/it]/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 33%|███████████████████████████████████████████████████                                                                                                      | 6/18 [00:22<01:22,  6.84s/it]

new best:  1.0 l1 0.01 355


/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 39%|███████████████████████████████████████████████████████████▌                                                                                             | 7/18 [00:37<01:43,  9.43s/it]/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 44%|████████████████████████████████████████████████████████████████████                                                                                     | 8/18 [00:51<01:51, 11.14s/it]/home/rob/Data/JACK_HACKATHON/hackathon/myenv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warning

In [ ]:
stronger_results = linear.main('./data/pair_attacks.npz')

In [16]:
coef = results[0].coef_

In [22]:
coef.shape

(1, 65536)

In [84]:
topk_idx = np.argsort(np.abs(coef)[0])[::-1][:10]
topk_vals = coef[0,topk_idx]
needed_idxs = coef.nonzero()[1]

In [85]:
import time
import pandas as pd
# load mapping
import goodfire
from tqdm import tqdm
client = goodfire.Client(api_key=api_key)

df = pd.read_csv('./feature_indexes_to_ids_map.csv')

def get_uuid_from_index(idx, model='meta-llama/Meta-Llama-3-8B-Instruct'):
    # check that `case` matches model, and index_in_sae matches idx, then return the `id` column
    return df[(df['case'] == model) & (df['index_in_sae'] == idx)]['id'].values[0]

def get_labels_from_indexs(idxs, batch_size=100):
    uuids = [get_uuid_from_index(idx) for idx in idxs]
    batched_uuids = [(uuids[i:i+batch_size], idxs[i:i+batch_size]) for i in range(0, len(uuids), batch_size)]
    labels = {}
    for batch_uuids, batch_idxs in tqdm(batched_uuids):
        features = client.features.list(batch_uuids)
        time.sleep(0.5)
        for idx, v in zip(batch_idxs, features._features.values()):
            labels[idx] = v.label
    return labels

labels = get_labels_from_indexs(needed_idxs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.14it/s]


In [87]:
topk_idx

array([17770, 26989, 41459, 29882, 51506,  8653, 43824, 47149, 13674,
       54472])

In [89]:
[labels[idx] for idx in topk_idx]

["AI personalizing response to user's needs",
 'Precise or emphatic adverbs and modifiers',
 'Fiction and fictional writing',
 'Opening parenthesis for citations and references',
 'The model offers to help the user',
 'The model indicates something is unnecessary or not needed',
 'The model is providing a list of options or items',
 'Linguistic phrases and expressions',
 'The model emphasizes sole or pure basis of something',
 'Quotation marks delimiting allowed words in constrained language tasks']

In [32]:
topk_idx

array([17770, 26989, 41459, 29882, 51506,  8653, 43824, 47149, 13674,
       54472])

In [50]:
# load Neel nanda pile 10k
from datasets import load_dataset

ds = load_dataset('NeelNanda/pile-10k')

README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

In [ ]:
resp = client.features.search(ds['train'][0]['text'][:100])

(FeatureGroup([
    0: "Simple indie or casual mobile game concepts",
    1: "Interactive games and quizzes",
    2: "Game over or win condition detection",
    3: "Game play moves and actions",
    4: "The model should complete a code snippet for a simple game",
    5: "The model should complete a code snippet for the game Pong",
    6: "Action of submitting or turning in content",
    7: "Adventure and survival game themes",
    8: "Child-oriented mobile apps and games",
    9: "Use of 'that' to introduce subordinate clauses or additional information"
 ]),
 [0.4608382284641266,
  0.4347690939903259,
  0.4068910777568817,
  0.4028874337673187,
  0.39343610405921936,
  0.392151415348053,
  0.39035332202911377,
  0.37867432832717896,
  0.37514179944992065,
  0.36837878823280334])

In [80]:
client.features.list(['56b9533a-fb87-43e9-8aa3-f5da292be286'])._features.values()

odict_values([Feature("Simple indie or casual mobile game concepts")])